In [ ]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold

credit 을 구하는 예제였음!

# 1.

데이터 분리는 StratifiedKFold 를 사용하여 y값 분포를 비슷하게 분리시킴. -> 5-fold

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in skf.split(train, train['credit']):
    folds.append((train_idx, valid_idx))

In [ ]:
random.seed(42)
lgb_models={}
for fold in range(5):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                         train['credit'][train_idx].values, train['credit'][valid_idx].values 
    lgb = LGBMClassifier(n_estimators=1000)
    lgb.fit(X_train, y_train, 
            eval_set=[(X_train, y_train), (X_valid, y_valid)], 
            early_stopping_rounds=30,
           verbose=100)
    lgb_models[fold]=lgb
    print(f'================================================================================\n\n')

In [ ]:
submit.iloc[:,1:]=0
for fold in range(5):
    submit.iloc[:,1:] += lgb_models[fold].predict_proba(test)/5

In [ ]:
submit.to_csv('submit/20210405_test_submit_ensemble.csv', index=False)

----------------------------------------------------------------------------------------------------------------------------------------------------------

# 2.

In [ ]:
X = train.drop(['credit'],axis=1)
y = train[['credit']]

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=100)
folds=[]
for train_idx, valid_idx in skf.split(X, y):
    folds.append((train_idx, valid_idx))

In [ ]:
lgb_models={}
lgb_outcomes=[]

for fold in range(5):
    train_idx, valid_idx = folds[fold]
    X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
    y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
    
    lgb = LGBMClassifier(n_estimators=2000, 
                         learning_rate=0.005, 
                         colsample_bytree=0.3,
                         num_leaves=300,
                         max_depth=38,
                         scale_pos_weight=5) 

    lgb.fit(X_train, y_train, 
            eval_set=[(X_train, y_train), (X_valid, y_valid)], 
            early_stopping_rounds=30,
            verbose=100)
    
    predictions = lgb.predict_proba(X_valid)
    logloss = log_loss(to_categorical(y_valid['credit']), predictions)
    lgb_outcomes.append(logloss)
    
    lgb_models[fold]=lgb   